# Snuffelfiets kwartaalrapportages

## Settings.

In [ ]:
# Generieke imports, variabelen en functies.

from pathlib import Path

import numpy as np
import pandas as pd

from snuffelfiets import inlezen, opschonen, analyse, plotting

api_key = ''  # voeg hier de API key toe

data_directory = Path('~','kwartaalrapportage').expanduser()

# date range selection settings
quarter = 'Q1'
year = '2023'
quarters = {
    'Q1': [1, 2, 3], 
    'Q2': [4, 5, 6], 
    'Q3': [7, 8, 9], 
    'Q4': [10, 11, 12], 
}
yq = f'{year}_{quarter}'

# data processing settings
error_code_selection = []
rit_splitter_interval = 1800  # s
ritfilters = dict(
    min_measurements=2,     # #
    max_duration=360,       # minutes
    max_distance=200,       # kilometers
    min_average_speed=1,    # km/h
    max_average_speed=35,   # km/h
    )
threshold_pm2_5 = 100

# mapbox settings (in degrees latlon)
b = {
    'N': [52.303634, 5.013507],
    'Z':  [51.858631, 5.040462],
    'O':  [51.954780, 5.627990],
    'W':  [52.226808, 4.794457],
}
mapbox_center = {
    'lat': b['Z'][0] + 0.5 * (b['N'][0] - b['Z'][0]),
    'lon': b['W'][1] + 0.5 * (b['O'][1] - b['W'][1]),
}
mapbox_extent = 1
hexagon_size = 0.010
hexbin_args = {
    'agg_func': np.nanmean,
    'color_continuous_scale': plotting.discrete_colorscale(),
    'range_color': [0, 100],
    'min_count': 2,
    'animation_frame': None,
    'width': 1920,
    'height': 1080,
    'opacity': 1.0,
    'zoom': 10,
    'center': mapbox_center,
    }
# add Utrecht polygons
# Import Utrecht province and township polygons
provincies, gemeenten = plotting.get_borders_utrecht(data_directory)
mapbox_layers = [{
    "name": "Gemeenten",
    "below": 'traces',
    "sourcetype": "geojson",
    "type": "line",
    "color": "gray",
    "source": gemeenten,
    }, 
    {
    "name": "Provincies",
    "below": 'traces',
    "sourcetype": "geojson",
    "type": "line",
    "color": "red",
    "source": provincies,
    }]
layout_args = {
    'mapbox_style': 'open-street-map',
    'coloraxis_showscale': False,
    'mapbox_layers': mapbox_layers,
    }

# Directories
output_directory = Path(data_directory, yq)
output_directory.mkdir(parents=True, exist_ok=True)

print(f'Analysing quarter {yq}; writing output to {output_directory}.')


## Data preparation

In [ ]:
# Read data from the database.  # NOTE: reading in monthly chunks because of serverside errors when trying to pull the full quarter

dfs = []
for i in quarters[quarter]:
    start_datum = f'{year}-{i:02d}-01'
    stop_datum = f'{year+1}-01-01' if i==12 else f'{year}-{i+1:02d}-01'
    df = inlezen.call_api(api_key, start_datum, stop_datum)
    dfs.append(df)
df = pd.concat(dfs)

filename = f'api_gegevens_{yq}.pickle'
p = Path(output_directory, filename)
df.to_pickle(p)

print(f'Read {df.shape[0]} measurements; saved raw data to {filename}.')


In [ ]:
# Drop the errors.
df = opschonen.verwijder_errors(df, error_codes=error_code_selection)


In [ ]:
# Correct units of selected columns.
df = opschonen.correct_units(df)


In [ ]:
# Convert timestamps to datetime objects and add dt columns.
df = analyse.bewerk_timestamp(df, split=True)


In [ ]:
# Split measuremnts into rides and add cycle stat columns.
df = analyse.split_in_ritten(df, t_seconden=rit_splitter_interval)


In [ ]:
# Filter the rides.
df = analyse.filter_ritten(df, **ritfilters)


In [ ]:
# Limiteer de PM2.5 waardes. FIXME: er is een betere methode nodig
df['pm2_5'][df['pm2_5'] >= threshold_pm2_5] = np.nan


## Summary Snuffelfiets statistics.

In [ ]:
def printfun(period, sumstats):

    print(f'\n==== {period} ====\n')

    print(f"Aantal fietsers: {sumstats['fietsers']['N']}\n")

    print(f"{' ':20} {'totaal':12} {'gemiddeld':12} {'topper':12}")
    print(f'-' * 56)
    stat = 'uren'
    print(f"FIETSTIJD [uur]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'afstand'
    print(f"AFSTAND    [km]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'ritten'
    print(f"RITTEN      [#]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")



In [ ]:
# Print the summary statistics for the quarter.
sumstats = analyse.summary_stats(df)
printfun(quarter, sumstats)


In [ ]:
# Print the summary statistics for the months in the quarter.
for i, dfm in df.groupby('month'):
    sumstats = analyse.summary_stats(dfm)
    printfun(f'2023{i:02d}', sumstats)


## Hexbin plots

In [ ]:
# Remove datapoints outside of the map area 
#   because it would take a very long time to process large areas.
#   TODO: doe dit op ritniveau (verwijderen ritten deels of geheel buiten de target area)
latlon = {
    'latitude': {'center': mapbox_center['lat'], 'extent': mapbox_extent},
    'longitude': {'center': mapbox_center['lon'], 'extent': mapbox_extent},
}
df = opschonen.filter_lat_lon(df, latlon)

# Plot the data for each month.
for i, dfm in df.groupby('month'):

    yyyymm = f'{year}{i:02d}'
    hexbin_args['title'] = yyyymm
    fig = plotting.hexbin_mapbox(dfm, hexagon_size, hexbin_args, layout_args)

    # Save image
    filestem = f'{yyyymm}_hexbin'
    output_stem = Path(output_directory, filestem)
    fig.write_html(f"{output_stem}.html")
    fig.write_image(f"{output_stem}.png")
    fig.write_image(f"{output_stem}.pdf")


## Uitgelicht 2023-Q1: Fijnstof en regen.

In [ ]:
# Specify the date range.
import calendar
months = quarters[quarter]
lastday = calendar.monthrange(int(year), quarters[quarter][2])[1]
dt_min = f'{year}-{months[0]}-1 00:00:00'
dt_max = f'{year}-{months[2]}-{lastday} 23:59:59'

# Import the weather data.
dfr = analyse.import_knmi_data(dt_min, dt_max, interval='dag', stations=[260], variables=['RH'])
dfr.RH = dfr.RH.values / 10  # RH: Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)


In [ ]:
# Calculate the mean daily PM2.5 value.
sfday = df.loc[:, ['pm2_5', 'day', 'month', 'year']].groupby(['day', 'month', 'year']).mean()
# Add to the weather data frame.
dfr['pm2_5'] = np.array(sfday['pm2_5'])


In [ ]:
# Plot the data.
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(dpi=300)
ax2 = ax1.twinx()

ax1.plot(dfr.index, dfr.pm2_5, c='C1')
ax2.plot(dfr.index, dfr.RH, c='C0')

ax1.set_ylabel('PM2.5 [ug/m3]', color='C1')
ax2.set_ylabel('Etmaalsom van de neerslag [mm]', color='C0')

ax1.plot([dfr.index[0], dfr.index[-1]], [25, 25], c='C1',linestyle='--')
plt.text(0.86, 0.56, 'norm', c='C1', transform=ax1.transAxes)
ax1.xaxis.set_tick_params(rotation=45)
plt.grid(linestyle=':')
